# Identificação das pessoas físicas associadas a cada processo gerando novo dataset.

Produtos:<br>
05 - Lista de Pessoas Fisicas ProInfo.xlsx<br>

Essa lista contem o nome de todas as pessoas físicas registradas como gestores no sistema ProInfo.

In [1]:
import pandas as pd
import regex as re

ModuleNotFoundError: No module named 'regex'

In [5]:
path_produtos = 'Dados\Produtos\\'
path_originais = 'Dados\Coletados\\'
path_ajustados = 'Dados\Ajustados\\'

# Extração do Nome das Pessoas Físicas cadastradas no campo "Envolvido Pessoa Física"

In [6]:
# Carrega DataSet gerado em "01 - Tratamento Inicial dos DataSets"
df_proinfo = pd.read_excel(path_produtos+'01 - ProInfo - Lista Consolidada v1_0.xlsx', index_col=0)

In [7]:
df_proinfo.columns

Index(['Colegiado', 'Número', 'Ano', 'Protocolo', 'Natureza',
       'Tipo de julgamento', 'Data da sessão', 'Relator', 'Exercício',
       'Situação', 'Envolvido Pessoa Física', 'Envolvido Pessoa Jurídica',
       'Nº de Origem'],
      dtype='object')

In [8]:
# Verifica como as informações estão estruturadas no campo "Envolvido Pessoa Física "
print(df_proinfo['Envolvido Pessoa Física'][-10:].to_markdown())

|      | Envolvido Pessoa Física                                                                                                                                                                                                                                                                                                         |
|-----:|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| 2515 | Notificado PF: ROBERTA SILVA DE CARVALHO SANTANA, Notificado PF: ADELSON DE ARAUJO PRATA                                                                                                                                                                                                                                        |
| 2516 

In [9]:
# Identifica os tipos de pessoas fisicas existentes no campo "Envolvido Pessoa Física"
separadores = []
for ind in df_proinfo.index:
    palavras = str(df_proinfo['Envolvido Pessoa Física'][ind]).split(' ')
    separadores.extend([palavra for palavra in palavras if ':' in palavra])

pd.DataFrame(separadores).value_counts()


Gestor:          7871
PF:              2641
Secretário:      1522
Responsável:     1123
Advogado:         916
Geral:             96
Estado:            57
Procuradoria:      45
Juiz(a):            6
Conselheiro:        2
Promotor:           1
Remetente:          1
Name: count, dtype: int64

In [44]:
# Gera um DataFrame que relaciona o nome e o tipo de cada gestor com o Protocolo.
import pandas as pd
import re

# Função para extrair nomes e tipo (Gestor ou Secretário) usando expressões regulares
def extrair_nomes_tipos(registro):
    padrao = re.compile(r'(Gestor|PF|Secretário|Responsável|Advogado|Geral|Estado|Procuradoria|Juiz(a)|Conselheiro|Promotor|Remetente):\s*([^,]+)')
    matches = padrao.findall(registro)
    return matches

def SeparaTiposPessoaFísica(df):

    # Aplicar a função à coluna 'Gestores' para criar novas colunas 'Nome' e 'Tipo'
    df['Nomes_Tipos'] = df['Envolvido Pessoa Física'].apply(extrair_nomes_tipos)

    # Explodir a lista de tuplas para várias linhas
    df_explodido = df.explode('Nomes_Tipos')

    # Dividir a tupla em duas colunas 'Nome' e 'Tipo'
    df_explodido[['Tipo','vazio', 'Nome']] = pd.DataFrame(df_explodido['Nomes_Tipos'].tolist(), index=df_explodido.index)

    # Limpar espaços extras nos nomes
    df_explodido['Nome'] = df_explodido['Nome'].str.strip()

    # Selecionar colunas desejadas no novo DataFrame
    return df_explodido[['Protocolo', 'Nome', 'Tipo']]


df_pf = SeparaTiposPessoaFísica(df_proinfo[['Protocolo','Envolvido Pessoa Física']].copy())
df_pf

,Protocolo,Nome,Tipo
0,TCE/016947/2002,MARIA PATRÍCIA DE SABÓIA ORRICO,Gestor
0,TCE/016947/2002,JAMES SILVA SANTOS CORREIA,Secretário
1,TCE/000684/2003,JAMES SILVA SANTOS CORREIA,Secretário
1,TCE/000684/2003,MARCOS DE MAGALHÃES TOURINHO,Gestor
1,TCE/000684/2003,MARIA PATRICIA DE SABOIA ORRICO,Gestor
...,...,...,...
2523,TCE/010710/2023,VICENTE JOSE DE LIMA NETO,PF
2524,TCE/008559/2021,WILSON JOSE VASCONCELOS DIAS,Gestor
2524,TCE/008559/2021,JERONIMO RODRIGUES SOUZA,Gestor
2524,TCE/008559/2021,CLEONICE MARIA SANTOS,Gestor


In [45]:
df_pf['Tipo'].value_counts()

Tipo
Gestor          7871
PF              2641
Secretário      1522
Responsável     1123
Advogado         916
Geral             96
Estado            57
Procuradoria      45
Conselheiro        2
Promotor           1
Remetente          1
Name: count, dtype: int64

In [46]:
# Existem dois tipos não esperados: "Estado" e "Geral"
print(df_pf[df_pf['Tipo']=='Estado'].head(6))
print(df_pf[df_pf['Tipo']=='Geral'].head(16))

          Protocolo                 Nome    Tipo
18  TCE/008485/2003   Luciane Rosa Croda  Estado
19  TCE/003999/2006   Luciane Rosa Croda  Estado
26  TCE/005138/2008  Aline Azevedo Nunes  Estado
30  TCE/004317/2007   Luciane Rosa Croda  Estado
30  TCE/004317/2007  Aline Azevedo Nunes  Estado
40  TCE/005210/2009  Aline Azevedo Nunes  Estado
           Protocolo                           Nome   Tipo
5    TCE/001381/2007  WELLINGTON CESAR LIMA E SILVA  Geral
25   TCE/002446/2008            Aline Azevedo Nunes  Geral
33   TCE/001516/2006             Luciane Rosa Croda  Geral
36   TCE/001331/2010             Luciane Rosa Croda  Geral
77   TCE/001534/2010            Aline Azevedo Nunes  Geral
87   TCE/003243/2011            Aline Azevedo Nunes  Geral
89   TCE/002283/2006                RUI MORAES CRUZ  Geral
89   TCE/002283/2006  WELLINGTON CESAR LIMA E SILVA  Geral
89   TCE/002283/2006     MARCIO JOSE CORDEIRO FAHEL  Geral
135  TCE/001385/2007     MARCIO JOSE CORDEIRO FAHEL  Geral
149  TCE

In [47]:
print(df_pf[df_pf['Nome']=='RUI MORAES CRUZ'])

            Protocolo             Nome         Tipo
89    TCE/002283/2006  RUI MORAES CRUZ        Geral
195   TCE/000150/2008  RUI MORAES CRUZ        Geral
208   TCE/000866/2002  RUI MORAES CRUZ        Geral
329   TCE/008388/2005  RUI MORAES CRUZ        Geral
1409  TCE/003128/2010  RUI MORAES CRUZ       Gestor
1457  TCE/003129/2010  RUI MORAES CRUZ  Responsável
1503  TCE/001352/2016  RUI MORAES CRUZ       Gestor


In [48]:
# Elimina os tipos de pessoas físicas que não são Convenentes nem Concedentes
# Os campos do tipo Geral e Estado podem ser qualquer coisa, portanto serão mantidos.
tipos_validos = ['Gestor', 'PF', 'Secretário', 'Responsável', 'Geral', 'Estado']
df_pf = df_pf[df_pf['Tipo'].isin(tipos_validos)].copy()
df_pf['Tipo'].value_counts()

Tipo
Gestor         7871
PF             2641
Secretário     1522
Responsável    1123
Geral            96
Estado           57
Name: count, dtype: int64

In [49]:
# Transforma todos os nomes em letras maiúsculas para facilitar comparações.
df_pf['Nome'] = [nome.upper() for nome in df_pf['Nome']]

In [51]:
df_pf.to_excel(path_produtos+'05 - Lista de Pessoas Fisicas ProInfo v1_0.xlsx')